In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Aditya\\Documents\\Aditya\\Hackathons\\init_prep\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Aditya\\Documents\\Aditya\\Hackathons\\init_prep'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class GenerateReportConfig:
    root_dir: Path
    model_path: Path
    report_path: Path

In [6]:
from heartDiseaseClassification.constants import *
from heartDiseaseClassification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_generate_report_config(self) -> GenerateReportConfig:
        config = self.config.generate_report

        create_directories([config.root_dir])

        generate_report_config = GenerateReportConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            report_path=config.report_path
        )

        return generate_report_config

In [8]:
from pathlib import Path
import joblib
from sklearn.metrics import classification_report
import pandas as pd

class GenerateReport:
    def __init__(self, config: GenerateReportConfig):
        self.config = config

    def generate_report(self):
        full_model = joblib.load(self.config.model_path)
        root_dir = Path(self.config.root_dir)
        X_train_scaled = pd.read_csv(f"{root_dir.parent}/data_ingestion/X_train_scaled.csv") 
        y_train = pd.read_csv(f"{root_dir.parent}/data_ingestion/y_train.csv")
        y_pred = full_model.predict(X_train_scaled)
        report = classification_report(y_train, y_pred)

        with open(self.config.report_path, "w") as f:
            f.write(report)
    

try:
    config = ConfigurationManager()
    generate_report_config = config.get_generate_report_config()
    generate_report = GenerateReport(config=generate_report_config)
    generate_report.generate_report()
except Exception as e:
    raise e

[2024-02-06 02:31:45,330: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-06 02:31:45,332: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-06 02:31:45,332: INFO: common: created directory at: artifacts]
[2024-02-06 02:31:45,334: INFO: common: created directory at: artifacts/generate_report]


C:\Users\Aditya\AppData\Local\Temp\ipykernel_14868\4037462586.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
